In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import cvxpy as cp
import pandas as pd
from matplotlib import cm
import matplotlib.pyplot as plt

from scipy import stats 
from scipy import interpolate
from sprd import grids, opt

In [3]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision = 4)
%matplotlib widget

In [4]:
npts = 101
save_figures = True

In [ ]:
# this_folder = os.path.dirname(os.path.abspath(__file__))
# parent_folder = os.path.dirname(this_folder)
# data_folder = os.path.join(this_folder, './data/')
file_x = os.path.join('./data', 'spread_env_x_2_density_01.csv')
file_y = os.path.join('./data', 'spread_env_y_1_density_01.csv')
file_s = os.path.join('./data', 'spread_env_sp_1_density_01.csv')
dfx = pd.read_csv(file_x)
dfy = pd.read_csv(file_y)
dfs = pd.read_csv(file_s)

xmin = min(dfx['x'].min(), dfy['x'].min())
xmax = max(dfx['x'].max(), dfy['x'].max())
xgrid = np.linspace(xmin, xmax, npts, endpoint=True)

assert (np.linalg.norm(np.diff(xgrid, 2)) < 1e-6)
sgrid = np.concatenate([xgrid - xgrid[-1], xgrid[1:] - xgrid[0]])


xdens = interpolate.interp1d(dfx['x'], dfx['y'], kind = 'linear', fill_value='extrapolate')(xgrid)
ydens = interpolate.interp1d(dfy['x'], dfy['y'], kind = 'linear', fill_value='extrapolate')(xgrid)
sdens = interpolate.interp1d(dfs['x'], dfs['y'], kind = 'linear', fill_value='extrapolate')(sgrid)

xdens /= xdens.sum()
ydens /= ydens.sum()
sdens /= sdens.sum()

x0 = np.sum(xdens*xgrid)
y0 = np.sum(ydens*xgrid)

print(x0,y0)

print(sum(dfx['y']), sum(dfy['y']), sum(dfs['y']))



In [6]:

test_plots = False

if test_plots:
    plt.figure()
    plt.plot(dfx['x'], dfx['y'], label = 'x dens input')
    plt.plot(xgrid, xdens, '.' , label = 'x dens interp')

    plt.plot(dfy['x'], dfy['y'],label = 'y dens input')
    plt.plot(xgrid, ydens, '.' ,label = 'y dens interp')

    plt.plot(dfs['x'], dfs['y'],label = 's dens input')
    plt.plot(sgrid, sdens, '.' ,label = 's dens interp')

    plt.legend(loc='best')
    plt.show()

### Check if the problem is feasible
(unfeasible is good)

In [ ]:
# tolerance; equality makes this hard
eps = 1e-6

# 2d density
x = cp.Variable((npts,npts))
constraints = opt.get_spread_constraints(x, xdens, ydens, sdens, eps = eps)

obj = cp.Minimize(opt.get_objf_idx(x, xgrid, idx_strike=x0+y0))
# obj = cp.Maximize(opt.get_objf_idx(x, xgrid, idx_strike=x0+y0) - smooth_obj)
prob = cp.Problem(obj, constraints)
# res = prob.solve()
res = prob.solve( solver = 'SCIPY', scipy_options = {'method':'highs'})
pij = x.value
print(prob.status)
print(prob.value)

### Dual

In [ ]:
f = cp.Variable((npts,))
g = cp.Variable(npts)
h = cp.Variable(len(sdens))
dual_obj = cp.Maximize(f@xdens + g@ydens - h@sdens)

dual_constraints = []
dual_constraints.append(cp.abs(f) <= 1)
dual_constraints.append(cp.abs(g) <= 1)
# dual_constraints.append(cp.abs(h) <= 10)

for k in range(len(sdens)):
    for i in range(npts):
        for j in range(npts):
            if i + j == k:
                dual_constraints.append(h[k] >= f[i] + g[j])



dual_prob = cp.Problem(dual_obj, dual_constraints)

# res = dual_prob.solve(solver=cp.ECOS)
res = dual_prob.solve( solver = 'SCIPY', scipy_options = {'method':'highs'})
print(dual_prob.status)
print(dual_prob.value)


In [ ]:
# print values of the strategies
print("Value of marginal strategy", f.value@xdens + g.value@ydens)
print("Value of spread strategy", h.value@sdens)

In [10]:
test_plots = False

if test_plots:
    plt.figure()
    plt.plot(xgrid, f.value, label = 'f')
    plt.plot(xgrid, g.value, label = 'g')
    plt.plot(sgrid, h.value, label = 'h')
    plt.show()


~~~ Plot PDFs and payoffs for our arb

In [ ]:
# X
fig, ax1 = plt.subplots()
ax1.plot(xgrid, xdens, label = 'PDF, left scale')

ax2 = ax1.twinx()
ax2.plot(xgrid, f.value, '.', color = 'orange', label = 'Payoff, right scale')
plt.title('PDF and arb payoff for X')
ax1.legend(loc = 'lower left')
ax2.legend(loc = 'upper right')
plt.show()


if save_figures:
    fname = './figs/spread_arb_X_01.pdf'
    fig.savefig(fname, bbox_inches='tight')

# Y
fig, ax1 = plt.subplots()
ax1.plot(xgrid, ydens, label = 'PDF, left scale')

ax2 = ax1.twinx()
ax2.plot(xgrid, g.value, '.', color = 'orange', label = 'Payoff, right scale')
plt.title('PDF and arb payoff for Y')
ax1.legend(loc = 'lower left')
ax2.legend(loc = 'upper right')
plt.show()


if save_figures:
    fname = './figs/spread_arb_Y_01.pdf'
    fig.savefig(fname, bbox_inches='tight')


# S
fig, ax1 = plt.subplots()
ax1.plot(sgrid, sdens, label = 'PDF, left scale')

ax2 = ax1.twinx()
ax2.plot(sgrid, h.value, '.', color = 'orange', label = 'Payoff, right scale')
plt.title('PDF and arb payoff for S')
ax1.legend(loc = 'lower left')
ax2.legend(loc = 'upper right')
plt.show()


if save_figures:
    fname = './figs/spread_arb_S_01.pdf'
    fig.savefig(fname, bbox_inches='tight')

In [12]:
#create 2D versions of the solutions and constraints for plotting

# the grid
sx,sy = np.meshgrid(xgrid,xgrid,indexing = 'ij')

# the f and g functions
sf,sg= np.meshgrid(f.value,g.value,indexing = 'ij')

# This is the sum -- what we can achieve by trading in marginals only
sfg = sf+sg

# This is the payoff that is dominated by sf+sg, a function of the diagonals only
sh = np.zeros((npts, npts))
for i in range(npts):
    for j in range(npts):
        sh[i,j] = h.value[i + j]


In [ ]:

step = 5
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(sx[::step,::step], sy[::step,::step], sh[::step,::step], \
            marker = '.', alpha = 1.0, label = 'diagonal payoff h(s)')
surf = ax.plot_surface(sx[::step,::step], sy[::step,::step], sfg[::step,::step], 
                        cmap = 'Reds', alpha=0.7, label = 'payoff from marginals f(x)+g(y)')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Spread envelope arbitrage')

# some magic workaround for the legend() bug, see 
# https://stackoverflow.com/a/54994985/14551426
# does not seem to work
surf._facecolors2d = surf._facecolor3d
surf._edgecolors2d = surf._edgecolor3d

plt.legend(loc = 'best')
plt.show()

if save_figures:
    fname = './figs/spread_arb_XYS_01.pdf'
    fig.savefig(fname, bbox_inches='tight')


### Check that triangle arbitrage does not pick this up
My paper eq (2.1) 

In [14]:
call_x = grids.euro_option_val_grid(xgrid, xdens, opt_type = 'call')
call_y = grids.euro_option_val_grid(xgrid, ydens, opt_type = 'call')
putt_y = grids.euro_option_val_grid(xgrid, ydens, opt_type = 'put')
call_s = grids.euro_option_val_grid(sgrid, sdens, opt_type = 'call')



In [15]:
# plot to check
plot_figures = False
cutoff = 3e-5

if plot_figures:
    plt.figure()
    plt.plot(xgrid, call_x, label='call_x')
    plt.plot(xgrid, -cutoff*np.ones_like(call_x), '-.', label='cutoff')
    plt.legend(loc='best')
    plt.show()

    plt.figure()
    plt.plot(xgrid, call_y, label='call_y')
    plt.plot(xgrid, putt_y, label='put_y')
    plt.plot(xgrid, -cutoff*np.ones_like(call_x), '-.', label='cutoff')
    plt.legend(loc='best')
    plt.show()

    plt.figure()
    plt.plot(sgrid, call_s, label='call_s')
    plt.plot(sgrid, -cutoff*np.ones_like(call_s), '-.', label='cutoff')
    plt.legend(loc='best')
    plt.show()


In [16]:


# put everything into 2D
sx,sy = np.meshgrid(xgrid,xgrid,indexing = 'ij')
call_xx, call_yy = np.meshgrid(call_x,call_y,indexing = 'ij')
call_xx, putt_yy = np.meshgrid(call_x,putt_y,indexing = 'ij')

call_ss = np.zeros((npts,npts))
for i in range(npts):
    for j in range(npts):
        call_ss[i,j] = call_s[npts -1 + i-j]


In [17]:
# check the plot to see it is all cosher
plot_figures = False


if plot_figures:
    step = 1
    fig3 = plt.figure()
    ax2 = fig3.add_subplot(projection='3d')
    ax2.plot_surface(sx[::step,::step], sy[::step,::step], call_ss[::step,::step], cmap = 'Reds', alpha=1.0, label = 'marginal payoffs')
    plt.xlabel('x')
    plt.ylabel('y')

    # plt.legend(loc = 'upper right')
    plt.show()

In [ ]:
# Check for tri arb 
cutoff = 3e-5

# this would  be non-negative if the joint density existed (it does not). See if it holds the other way
triarb_gap_lhs = call_ss - call_xx + call_yy
triarb_gap_rhs = call_xx + putt_yy - call_ss

arb_detected_lhs = (triarb_gap_lhs < -cutoff*1e-3 ).any() # this is the important case make sure we do not miss it
arb_detected_rhs = (triarb_gap_rhs < -cutoff ).any() # here if cutoff is too small we get some weird boundary effects

print(f'LHS triangle arbitrage detected: {arb_detected_lhs}')
print(f'RHS triangle arbitrage detected: {arb_detected_rhs}')


In [19]:
# we have some weird boundary effects for rhs (which is NOT related to our primary investigation as 
# we are looking at the LHS). But for completeness let's investigate a bit more
if arb_detected_rhs:
    print(np.sum(triarb_gap_rhs*(triarb_gap_rhs < -cutoff)))
    print(np.sum(triarb_gap_rhs < -cutoff))

    idx_min = np.argmin(triarb_gap_rhs)
    imin,jmin = np.unravel_index(idx_min, (npts,npts))

    print(imin, jmin, call_xx[imin,jmin], putt_yy[imin, jmin], call_ss[imin, jmin], triarb_gap_rhs[imin,jmin])

    fig4 = plt.figure()
    ax2 = fig4.add_subplot(projection='3d')
    ax2.plot_surface(sx[::step,::step], sy[::step,::step], 
                    triarb_gap_rhs[::step,::step] * (triarb_gap_rhs[::step,::step] < -cutoff), cmap = 'Reds', alpha=1.0, label = 'marginal payoffs')
    plt.xlabel('x')
    plt.ylabel('y')


In [ ]:
step = 1

fig3 = plt.figure()
ax2 = fig3.add_subplot(projection='3d')
ax2.plot_surface(sx[::step,::step], sy[::step,::step], triarb_gap_lhs[::step,::step], cmap = 'Reds', alpha=1.0, label = 'marginal payoffs')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Triangle arbitrage, LHS gap')


fig4 = plt.figure()
ax2 = fig4.add_subplot(projection='3d')
ax2.plot_surface(sx[::step,::step], sy[::step,::step], 
                triarb_gap_rhs[::step,::step], cmap = 'Reds', alpha=1.0, label = 'marginal payoffs')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Triangle arbitrage, RHS gap')

if save_figures:
    fname = './figs/triangle_arb_01.pdf'
    fig.savefig(fname, bbox_inches='tight')

In [21]:
# dig a bit deeper into rhs
if arb_detected_rhs:
    x_coords = []
    y_coords = []
    xval = []
    yval = []
    sval = []

    for i in range(npts):
        for j in range(npts):
            if triarb_gap_rhs[i,j] < -cutoff:
                x_coords.append(i)
                y_coords.append(j)
                xval.append(call_xx[i,j])
                yval.append(putt_yy[i,j])
                sval.append(call_ss[i,j])


    print(list(zip(x_coords, y_coords)))
    print(list(zip(xval, yval, sval)))